In [1]:
from data_common.notebook import *
from data_common.dataset import get_dataset_df
from data_common.pandas.df_extensions.la import get_la_df
from datetime import date

council_date = date(2023, 4, 2)

# Composite IMD distance calculation

This notebook generates distance calculations and labels based on IMD scores for specific LSOAs, based on the [uk composite](https://github.com/mysociety/composite_uk_imd).

This works in two ways, labels are produced by creating overall scores for each authority via the method for creating scores for higher geographies. Labels for county/combined are generated by putting them into the deciles as worked out by the lower geographies (so the original deciles are not affected by counting lots of the population twice).

The actual distance works by creating three values for authority - the percentage of pop that lives in the 1, 2-3, and 4-5 deciles. 
This creates distance measures that are a bit more sensitive to different kinds of mixes of deprivation.

In [2]:
# bring in the uk-wide imd info

# split into three dimensions, decile 1 is high, 2,3 is medium, 4,5 is low.

imd = pd.read_csv(Path("data", "packages", "uk_index", "UK_IMD_E.csv")).set_index(
    "lsoa"
)
imd = imd[["UK_IMD_E_pop_quintile"]].rename(
    columns={"UK_IMD_E_pop_quintile": "quintile"}
)
imd["group"] = imd["quintile"].map(
    {
        1: "High deprivation",
        2: "Medium deprivation",
        3: "Medium deprivation",
        4: "Low deprivation",
        5: "Low deprivation",
    }
)

ruc = get_dataset_df("uk_ruc", "uk_ruc", "latest", "composite_ruc.csv",).set_index(
    "lsoa"
)[["pop"]]

imd = imd.join(ruc)
imd

,quintile,group,pop
lsoa,,,
95ZZ06W1,1,High deprivation,1662
95GG47S2,1,High deprivation,1758
95GG35S2,1,High deprivation,1399
95MM12S2,1,High deprivation,1211
95MM27S1,1,High deprivation,1825
...,...,...,...
E01017787,5,Low deprivation,1588
S01008861,5,Low deprivation,769
S01006581,5,Low deprivation,940


In [3]:
# get this into the form of a sheet where the percentage of each lower-tier la in different deprivation bands is in different columns
lsoa_lookup = get_dataset_df(
    "uk_local_authority_names_and_codes",
    "uk_la_past_current",
    "latest",
    "lookup_lsoa_to_registry.csv",
).set_index("lsoa")

# map of old to new
la_df = get_la_df(include_historical=True, as_of_date=council_date)
# just where there is a replaced-by
la_df = la_df[la_df["replaced-by"].notnull()]
# map of old to replaced code
old_to_new_lookup = la_df.set_index("local-authority-code")["replaced-by"].to_dict()

# update the lsoa mapping with any changes
lsoa_lookup["local-authority-code"] = lsoa_lookup["local-authority-code"].apply(
    lambda x: old_to_new_lookup.get(x, x)
)
lsoa_lookup = lsoa_lookup

In [4]:
df = imd.join(lsoa_lookup, how="outer")
pt = df.pivot_table(
    "pop", index="local-authority-code", columns=["group"], aggfunc="sum"
).fillna(0)

# add higher levels
hpt = (
    pt.reset_index()
    .la.to_multiple_higher(aggfunc="sum", as_of_date=council_date)
    .set_index("local-authority-code")
)
pt = pd.concat([pt, hpt])

# adding density in as another factor to pill apart similar deprivations
pt = (
    pt.common.row_percentages()
    .reset_index()
    .la.get_council_info(["pop-2020", "area", "official-name"], as_of_date=council_date)
)
pt["density"] = pt["pop-2020"] / pt["area"]
pt = pt.drop(columns=["area", "pop-2020"])
pt = pt.rename(columns=lambda x: x.lower().replace(" ", "-"))

pt = pt[
    [
        "local-authority-code",
        "low-deprivation",
        "medium-deprivation",
        "high-deprivation",
        "official-name",
        "density",
    ]
]
split_percent = pt

pt

,local-authority-code,low-deprivation,medium-deprivation,high-deprivation,official-name,density
0,ABC,0.00,0.39,0.61,"Armagh City, Banbridge and Craigavon Borough C...",162.36
1,ABD,0.75,0.23,0.02,Aberdeenshire Council,41.31
2,ABE,0.58,0.35,0.08,Aberdeen City Council,1231.51
3,ADU,0.51,0.47,0.02,Adur District Council,1528.26
4,AGB,0.45,0.47,0.08,Argyll and Bute Council,12.37
...,...,...,...,...,...,...
388,SCR,0.27,0.37,0.36,South Yorkshire Mayoral Combined Authority,911.76
389,TVCA,0.32,0.28,0.39,Tees Valley Combined Authority,851.79
390,WECA,0.50,0.36,0.14,West of England Combined Authority,996.89
391,WMCA,0.19,0.35,0.46,West Midlands Combined Authority,3259.34


In [5]:
# This follows methodology in https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/833947/IoD2019_Research_Report.pdf p. 69


# merge lsoa to la lookup column
df = pd.read_csv(Path("data", "packages", "uk_index", "UK_IMD_E.csv")).set_index("lsoa")
df = df.join(lsoa_lookup)

# merge lsoa population in
pop = get_dataset_df("uk_ruc", "uk_ruc", "latest", "composite_ruc.csv").set_index(
    "lsoa"
)[["pop"]]
df = df.join(pop)

# create a population adjusted score
df["pop_score"] = df["UK_IMD_E_score"] * df["pop"]

# pivot up to the local authority level
pt = df.pivot_table(["pop_score", "pop"], index="local-authority-code", aggfunc="sum")  # type: ignore

# combine these for the higher level authorities
pt = pt.reset_index()
higher_df = pt.la.to_multiple_higher(aggfunc="sum", as_of_date=council_date)

# calculate a new score, dividing the summed score by the summed population
pt["la-deprivation-score"] = pt["pop_score"] / pt["pop"]
higher_df["la-deprivation-score"] = higher_df["pop_score"] / higher_df["pop"]

# at this point we're calculating population quintiles based on lower tiers
df = pt.sort_values("la-deprivation-score", ascending=False)
df["cum_pop"] = df["pop"].astype("int").cumsum()
df["la-imd-pop-quintile"] = np.ceil(df["cum_pop"] / sum(df["pop"]) * 5).astype(int)
df["la-imd-pop-decile"] = np.ceil(df["cum_pop"] / sum(df["pop"]) * 10).astype(int)
df = df.drop(columns=["pop", "pop_score", "cum_pop"])

# now we slot in the higher tiers into the quintiles already established
higher_df["la-imd-pop-decile"] = 0
higher_df["la-imd-pop-quintile"] = 0
pt = (
    pd.concat([df, higher_df])
    .sort_values("la-deprivation-score", ascending=False)
    .set_index("local-authority-code")
)
last_quin = 1
last_dec = 1
for i, row in pt.iterrows():
    if row["la-imd-pop-quintile"] == 0:
        pt.loc[i, "la-imd-pop-quintile"] = last_quin  # type: ignore
    else:
        last_quin = row["la-imd-pop-quintile"]
    if row["la-imd-pop-decile"] == 0:
        pt.loc[i, "la-imd-pop-decile"] = last_dec  # type: ignore
    else:
        last_dec = row["la-imd-pop-decile"]

pt = pt.drop(columns=["pop", "pop_score"])
pt.head()

,la-deprivation-score,la-imd-pop-quintile,la-imd-pop-decile
local-authority-code,,,
DRS,62.31,1,1
BFS,55.49,1,1
FMO,48.63,1,1
NMD,48.62,1,1
CCG,46.68,1,1


In [6]:
# get the quintiles as labels


def num_to_ith(num: Union[float, int]) -> str:
    """1 becomes 1st, 2 becomes 2nd, etc."""
    value = str(num)
    before_last_digit = 0
    last_digit = value[-1]
    if len(value) > 1 and before_last_digit == "1":
        return value + "th"
    if last_digit == "1":
        return value + "st"
    if last_digit == "2":
        return value + "nd"
    if last_digit == "3":
        return value + "rd"
    return value + "th"


def label_quintile(v: float) -> str:
    label = f"{num_to_ith(int(v))} IMD quintile"
    return label


df = pt.reset_index()

df["label"] = df["la-imd-pop-quintile"].apply(label_quintile)

# move raw columns to end to preserve versioning

df.to_csv(Path("data", "la_labels.csv"), index=False)
df

,local-authority-code,la-deprivation-score,la-imd-pop-quintile,la-imd-pop-decile,label
0,DRS,62.31,1,1,1st IMD quintile
1,BFS,55.49,1,1,1st IMD quintile
2,FMO,48.63,1,1,1st IMD quintile
3,NMD,48.62,1,1,1st IMD quintile
4,CCG,46.68,1,1,1st IMD quintile
...,...,...,...,...,...
388,MSS,7.74,5,10,5th IMD quintile
389,WAE,7.51,5,10,5th IMD quintile
390,RUS,7.20,5,10,5th IMD quintile
391,WOK,5.83,5,10,5th IMD quintile


In [7]:
# make the label description

labels = pd.Series(df["label"].unique()).to_frame().rename(columns={0: "label"})

labels["desc"] = [
    "Councils in most deprived quintile (20%)",
    "Councils in second most deprived quintile (20%)",
    "Councils in middle deprivation quintile (20%)",
    "Councils in second least deprived quintile (20%)",
    "Councils in least deprived quintile (20%)",
]
final = df.merge(labels, on="label").merge(split_percent, on="local-authority-code")

end_columns = ["la-imd-pop-quintile", "la-imd-pop-decile"]
start_column = ["local-authority-code", "official-name"]
cols = [x for x in final.columns if x not in end_columns + start_column]
final = final[start_column + cols + end_columns]


final.to_csv(Path("data", "packages", "uk_index", "la_imd.csv"), index=False)
final.head()

,local-authority-code,official-name,la-deprivation-score,label,desc,low-deprivation,medium-deprivation,high-deprivation,density,la-imd-pop-quintile,la-imd-pop-decile
0,DRS,Derry City and Strabane District Council,62.31,1st IMD quintile,Councils in most deprived quintile (20%),0.00,0.14,0.86,120.79,1,1
1,BFS,Belfast City Council,55.49,1st IMD quintile,Councils in most deprived quintile (20%),0.02,0.31,0.67,2482.32,1,1
2,FMO,Fermanagh and Omagh District Council,48.63,1st IMD quintile,Councils in most deprived quintile (20%),0.00,0.05,0.95,40.97,1,1
3,NMD,"Newry, Mourne and Down District Council",48.62,1st IMD quintile,Councils in most deprived quintile (20%),0.00,0.18,0.82,108.01,1,1
4,CCG,Causeway Coast and Glens Borough Council,46.68,1st IMD quintile,Councils in most deprived quintile (20%),0.00,0.19,0.81,72.98,1,1
